# Introducción
En el presente informe buscamos exponer el avanze realizado en la exploración, refinamiento y, finalmente, las respuestas a las Preguntas planteadas en el Hito 1.
Para esto, cada integrante se centro en una de ellas, estableciendo una metodología para responder a cada una, para luego, en conjunto desarrollar la implementación de una de estos métodos.

#### Enseguida exponemos cómo fueron replanteadas y refinadas nuestras Preguntas:

1. ¿Se puede predecir las características de las matrículas para años futuros considerando los datos actuales?
2. ¿Cuáles son las características y tendencias de las personas que se cambian de carrera? -> caracteriza las carreras con mayor tendencia a “cambio de carrera” y, a la vez, a las personas que tienen tendencia a “cambio de carrera”
3. ¿Cuánto valor le agrega la acreditación a una carrera?
4. ¿Existe una relación entre costo de carrera y otras variables (numéricas?)?
 
Para responder las preguntas primero se preprocesan los datos, tal como se indica en la sección de Análisis Exploratorio, para no tener problemas de compatibilidad en la representación de ciertos valores, como es el caso de los datos nulos.

#### Pregunta 1:

Antes de esto hay que replantearse la pregunta, y para esto se debe definir concretamente cuáles son las características de las matrículas. En el caso de que fueran instancias, es decir, cada matrícula ingresada, entonces esta pregunta sería equivalente a la pregunta dos: debemos predecir características en personas, aunque en este caso no nos estaríamos centrando en personas que se cambian de carrera.
En el caso de que fueran características macroscópicas, se deben evitar preguntas triviales, que involucren directamente indicadores como promedios, varianzas, número de sucesos, etc., que debieron ser estudiadas en el hito anterior. Una alternativa interesante podría ser generar clusters con los vectores totales (luego aplicar one hot) de cada instancia. y ver su evolución temporal: ¿dónde está el centro de cada uno, por año?, ¿cómo van cambiando ciertos indicadores de validación: Inercia, Separación, Cohesión,?, ¿cómo va cambiando la densidad de clusters?. Junto a esto podemos ir comparando con valores más tangibles como promedios, varianzas, etc. (ej: varianza regional del valor de arancel).

Para responder esta pregunta, se deben completar tres pasos previos:
- Utilizar nuevamente como vectorización preliminar one hot encoding para ingresar las instancias como vectores de dimensión igual al número de columnas. Habrá que tener cuidado interpretando los clusters y el espacio vectorial en el que residen. En este vector, por ejemplo, la columna de rango de edad (ej: 18 y 20 años) tendría igual peso que la del tipo de estudio superior (ej: Doctorado); los dos marcarían un 1 en su dimensión correspondiente, y en las demás, de su misma naturaleza, un 0.

- Generar Clustering con ciertos atributos seleccionados gracias a la exploración realizada en el hito 1. Con esto, y la correspondiente validación del clustering, podremos cuantificar potenciales patrones de agrupación.
    - Primeramente se utilizará el coeficiente de Silhouette para verificar la viabilidad del cluster, si fuese negativo, se trataría de generar un vector (por instancia) preliminar compuesto de otra manera, o también podría ser interesante crear para cada columna un vector y comprobar si podemos, mediante algún tipo de operación, unificarlos todos, de forma que el clustering de cuenta de estructuras particulares.
    - Se hará uso de la métrica Cohesión, Separación e Inercia, junto a la visualización de su matriz de proximidad, una vez encontrado un coeficiente de Silhouette positivo, con el propósito de seguir evaluando las estructuras subyacentes en los datos. 
    - Podemos tomar este proceso como una forma sistemática de vectorizar instancias, de modo que luego quedar satisfechos (o de plano probar con varias) con un encoding podremos proseguir al paso 3.

- Luego de tener un vector para cada instancia, podemos reducir su dimensionalidad y  visualizar los clusters, como primera opción mediante PCA, pero existen varias por explorar. Luego de esto tendremos instancias medianamente interpretables, y que facilitarán su manejo en algoritmos de Regresión. Como ya se dijo en un inicio, otro posible estudio es el de la dinámica de los cluster, sus propiedades (ej: su densidad), y la correlación que estas variables guardan con indicadores macroscópicos en las matrículas(ej: valor arancel medio, arancel medio por región, etc.).

#### Pregunta 2:
 
Para responder la pregunta 2 primero es importante notar que es posible determinar quienes se han cambiado de carrera con los datos actuales, por lo tanto, se utilizará clasificación para buscar perfiles de personas que podrían cambiarse de carrera en el futuro. Así, las clases posibles son “se cambió de carrera” y “no se cambió de carrera”, y nuestra clase positiva corresponde a quienes sí se han cambiado de carrera, pues es lo que queremos evitar.

Entonces lo primero es disminuir la dimensionalidad de los datos, para esto se agregará una nueva columna que describe el tiempo que lleva la persona cursando su carrera actual, así esta resume la información de las columnas ``cat_periodo`` y ``ANIO_ING_CARR_ACT``, su creación sería a través de la resta de los años de ``ANIO_ING_CARR_ACT`` y ``cat_periodo``. Por otro lado, se requiere una columna que indique la clase de la fila, la que resumiría la información de las columnas ``ANIO_ING_CARR_ORI``, ``SEM_ING_CARR_ORI``, ``ANIO_ING_CARR_ACT`` y ``SEM_ING_CARR_ACT``; para obtenerla haremos una restricción con la siguiente query: [agregar query].
Adicionalmente se mantendremos las siguientes columnas;
- GEN_ALU
- RANGO_EDAD
- TIPO_INST_3
- JORNADA
- TIPO_PLAN_CARR
- DUR_ESTUDIO_CARR
- NIVEL_CARRERA_1 o NIVEL_GLOBAL
- VALOR_MATRICULA
- VALOR_ARANCEL
- OECD_AREA
- ACREDITADA_CARR o ACREDITADA_INST
- FORMA_DE_INGRESO

Donde se incluye una sola columna relacionada a la duración de la carrera pues se observa una estrecha relación entre DUR_ESTUDIO_CARR y DUR_TOTAL_CARR; una correlación de 0.99. A pesar de esto siguen siendo demasiadas columnas, por lo que aplicaremos la transformación One Hot sobre los datos restringidos y eliminaremos las columnas que tengan correlación cercana a 1 o a -1. 
Una vez determinadas las columnas a utilizar se busca encontrar el mejor algoritmo de clasificación, para lo cual se evaluarán las métricas de los algoritmos **KNN**, **Naive Bayes**, **Support Vector Machine** y **Árbol de Decisión**. Estas métricas serán evaluadas con la técnica **holdout**, dejando $1/3$ de los datos para entrenar y aplicando estratificación sobre las particiones, pero, además, se observará el desempeño aplicando **subsampling** sobre la clase negativa, **oversampling** sobre la clase positiva y el caso en que no se aplica ninguno, esto al considerar que son pocos los alumnos que se cambian de carrera anualmente. Luego, usaremos matriz de confusión para observar el desempeño, poniendo especial atención en la métrica **recall**, dado que no es un gran problema tener falsos positivos, pues lo que se busca es mejorar la información entregada a estudiantes previo a su primera matrícula, considerando quienes tienen un perfil con mayor probabilidad de deserción de su primera carrera.

#### Pregunta 3:

- Definir columnas numéricas a utilizar:
    - GEN_ALU
    - TIPO_INST_3


- Definir columnas categóricas a utilizar
 
- Definir qué es “valor”. Se busca definir métricas que den cuenta del valor agregado considerando 3 directrices:1. Demanda de los estudiantes para cada carrera. 2. Valor monetario del arancel. 3. Posicionamiento de la marca(acreditación sirve para estar entre universidades top)
- Preprocesamiento: Se realiza una agrupación (mediante clustering) de características para poder reducir la dimensionalidad del problema, junto con esto se hace una representación en one-hot de las características categóricas, lo que permitirá hacer otras visualizaciones.
- Evolución temporal: Se verá cómo evolucionan las métricas seleccionadas año tras año. Se generan visualizaciones implementadas en python (matplotlib) para este objetivo.

#### Pregunta 4:
 
Propuesta experimental:
- Dado que esta pregunta es de carácter descriptivo, se propone utilizar técnicas de clustering para encontrar automáticamente  si hay relaciones de forma natural entre la variable valor del arancel  y otras variables numéricas, como la duración de la carrera y los años de acreditación. 
- También se propone buscar relaciones del valor del arancel con variables categóricas como la modalidad (presencial, semi-presencial o no presencial) y el tipo de jornada (diurno, vespertino), realizando codificaciones numéricas a estas categorías para poder aplicar clustering.
- Se probarán técnicas de clustering particionales (K-Means) y jerárquicos.
- Se estimará el k en K-Means utilizando la métrica SSE.
- Se validará la calidad de los clustering usando la medida SSE, Matriz de similitud y el coeficiente de Silhouette. Se seleccionarán aquellos clustering que entreguen resultados más significativos.
- A partir de los resultados obtenidos, se generarán visualizaciones del clustering y dendrogramas para identificar posibles outliers y realizar un post- procesamiento de los datos en caso de ser necesario, con el objetivo de reducir el SSE.